In [1]:
prog = "1 + 1"

"1 + 1"

In [2]:
ex1 = parse(prog)

:(1 + 1)

In [3]:
typeof(ex1)

Expr

In [4]:
ex1.head

:call

In [5]:
ex1.args

3-element Array{Any,1}:
  :+
 1  
 1  

In [6]:
ex2 = Expr(:call, :+, 1, 1)

:(1 + 1)

In [7]:
ex1 == ex2

true

**Julia** code is a data structure that is accessible from the language itself.

In [8]:
dump(ex2)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 1
    3: Int64 1
  typ: Any


In [10]:
# nesting Expr
ex3 = parse("(4 + 4) / 2")

:((4 + 4) / 2)

In [11]:
Meta.show_sexpr(ex3)

(:call, :/, (:call, :+, 4, 4), 2)

# Symbols

In [12]:
:foo

:foo

In [13]:
typeof(ans)

Symbol

In [14]:
:foo == Symbol("foo")

true

In [15]:
Symbol("func", 10)

:func10

In [16]:
Symbol(:var, '_', "sym")

:var_sym

In [17]:
:(:)

:(:)

In [18]:
:(::)

:(::)

# Expressions

## Quoting

`:` creates expressions without using `Expr` explicitly.

**Julia** calls this *quoting*.

In [19]:
ex = :(a+b*c+1)

:(a + b * c + 1)

In [20]:
typeof(ex)

Expr

In [21]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((4,))
    1: Symbol +
    2: Symbol a
    3: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol *
        2: Symbol b
        3: Symbol c
      typ: Any
    4: Int64 1
  typ: Any


In [22]:
:(a + b*c + 1)  ==
    parse("a + b*c + 1") ==
    Expr(:call, :+, :a, Expr(:call, :*, :b, :c), 1)

true

In [24]:
ex = quote
    x = 1
    y = 2
    x + y
end

quote  # In[24], line 2:
    x = 1 # In[24], line 3:
    y = 2 # In[24], line 4:
    x + y
end

In [25]:
typeof(ex)

Expr

## Interpolation

*splicing* is done with the `$` prefix.

*Interpolate the literal value of `a`:

In [26]:
a = 1;

In [27]:
ex = :($a + b)

:(1 + b)

*Interpolate* the tuple (1,2,3) as an expression into a conditional test:

In [28]:
ex = :(a in $:((1,2,3)) )

:(a in (1, 2, 3))

In [30]:
:( :a in $( :(:a + :b) ) )

:(:a in :a + :b)

## `eval()`  and effects

**Julia** will execute at global scope using `eval()`:

In [31]:
:(1 + 2)

:(1 + 2)

In [32]:
eval(ans)

3

In [33]:
ex = :(a + b)

:(a + b)

In [34]:
eval(ex)

LoadError: [91mUndefVarError: b not defined[39m

In [35]:
a = 1; b = 2;

In [36]:
eval(ex)

3

In [37]:
a = 3; b = 4;
eval(ex)

7

In [38]:
a = eval(ex);

In [39]:
eval(ex)

11

Each [module]() has it's own `eval()` function. Expressions may have *side-effects* that alter the state of the module's environment:

In [41]:
ex = :(x = 1)

:(x = 1)

In [42]:
x

LoadError: [91mUndefVarError: x not defined[39m

In [43]:
eval(ex)

1

In [44]:
x

1

With **Julia**, we can dynamically generate and execute arbitrary code:

In [45]:
a = 1;

In [46]:
ex = Expr(:call, :+, a, :b)

:(1 + b)

In [47]:
a = 0; b = 2;

In [48]:
eval(ex)

3

In [49]:
function math_expr(op, op1, op2)
    expr = Expr(:call, op, op1, op2)
    return expr
end

math_expr (generic function with 1 method)

In [50]:
ex = math_expr(:+, 1, Expr(:call, :*, 4, 5))

:(1 + 4 * 5)

In [51]:
eval(ex)

21

In [52]:
function make_expr2(op, opr1, opr2)
    opr1f, opr2f = map(x -> isa(x, Number) ? 2*x : x, (opr1, opr2))
    retexpr = Expr(:call, op, opr1f, opr2f)
    return retexpr
end

make_expr2 (generic function with 1 method)

In [53]:
make_expr2(:+, 1, 2)

:(2 + 4)

In [54]:
ex = make_expr2(:+, 1, Expr(:call, :*, 5, 8))

:(2 + 5 * 8)

In [55]:
eval(ex)

42

## Macros

In [57]:
macro sayhello()
    return :( println("Hello, world!") )
end

@sayhello (macro with 1 method)

Compiler replaces all `@sayhello` with `:( println("Hello, world!") )`

When `@sayhello` is entered in the REPL, the expression executes immediately, so we only see the result:

In [58]:
@sayhello

Hello, world!


In [59]:
macro sayhello(name)
    return :( println("Hello, ", $name) )
end

@sayhello (macro with 2 methods)

In [60]:
@sayhello("human")

Hello, human


In [61]:
ex = macroexpand( :(@sayhello("human")))

:((Main.println)("Hello, ", "human"))

In [62]:
typeof(ex)

Expr

In [63]:
@macroexpand @sayhello "human"

:((Main.println)("Hello, ", "human"))